<a href="https://colab.research.google.com/github/python-freak/Custom-Layer-in-Keras/blob/master/DenseLayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras import backend as k
from keras.layers import Layer
from keras import activations , initializers 
from keras.engine import InputSpec
from keras.engine.base_layer import Layer
from keras.utils.generic_utils import deserialize_keras_object

In [0]:
class DenseLayer(Layer):
  
  def __init__(self,
             units,
             activation = None , 
             use_bias = True , 
             kernel_initializer = 'glorot-uniform' ,
             bias_initializer = 'zeros', **kwargs):
    
    
    if 'input_shape' not in kwargs and 'input_dim' in kwargs:
      kwargs['input_shape'] = (kwargs.pop('input_dim') , )
    
    super(DenseLayer , self).__init__(**kwargs)
    self.units = units
    self.activation = activations.get(activation)
    self.use_bias = use_bias
    self.kernel_initializer = initializers.get(kernel_initializer)
    self.bias_initializer = initializers.get(bias_initializer)
    self.input_spec = InputSpec(min_ndim = 2)
    self.supports_masking = True
    
    def build(self , input_shape):
      assert len(input_shape >= 2)
      
      input_dim = input_shape[-1]
      self.kernel = self.add_weight(shape=(input_dim,self.units) , initializer = self.kernel_initializer , name = 'kernel')
      if self.use_bias:
        self.bias = self.add_weight(shape=(self.units,) , initializer = self.bias_initializer , name = 'bias')
        
      self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
      self.built = True
      
      
  def call(self , inputs):
    output = k.dot(inputs , self.kernel)
    if self.use_bias:
      output = k.bias_add(output , self.bias , data_format = "channels_last")
    if self.activation is not None:
      output = self.activation(output)
      
    return output
  
  
  def compute_output_shape(self , input_shape):
    assert input_shape and len(input_shape)>=2
    assert input_shape[-1]
    output_shape = list(input_shape)
    output_shape[-1] = self.units
    return tuple(output_shape)
  
  def get_config(self):
    config = {
        'units':self.units , 
        'activation': activations.serialize(self.activation), 
        'use_bias': self.use_bias, 
        'kernel_initializer' :initializers.serialize(self.kernel_initializer) ,
        'bias_initializer':initializers.serialize(self.bias_initializer)
    }
    base_config = super(Dense , self).get_config()
    return dict(list(base_config.items()) + list(config.items()))
    
        
    
    